In [1]:
from ltr.client import ElasticClient
client = ElasticClient()

In [ ]:
from ltr import download
download();

In [ ]:
from ltr.index import rebuild_tmdb
rebuild_tmdb(client)

In [31]:
config = {"validation": {
              "index": "tmdb",
              "params": {
                  "keywords": "rambo"
              }    
           },
           "featureset": {
            "features": [
                { #1
                    "name": "title_bm25",
                    "params": ["keywords"],
                    "template": {
                        "match": {"title": "{{keywords}}"}
                    }
                },
                { #2
                    "name": "overview_bm25",
                    "params": ["keywords"],
                    "template": {
                        "match": {"overview": "{{keywords}}"}
                    }
                },
                { #3
                    "name": "tagline_bm25",
                    "params": ["keywords"],
                    "template": {
                        "match": {"tagline": "{{keywords}}"}
                    }
                },
                { #4
                    "name": "genre_bm25",
                    "params": ["keywords"],
                    "template": {
                        "match": {"genres": "{{keywords}}"}
                    }
                }
            ]
    }}

from ltr import setup
setup(client, config=config, index='tmdb', featureset='zenit-movies')

Removed Default LTR feature store [Status: 200]
Initialize Default LTR feature store [Status: 200]
Create zenit-movies feature set [Status: 201]


In [33]:
from ltr.log import judgments_to_training_set
trainingSet = judgments_to_training_set(client, 
                                        judgmentInFile='data/genome_judgments.txt', 
                                        trainingOutFile='data/genome_judgments_train.txt', 
                                        featureSet='zenit-movies')

Recognizing 1128 queries...
Parsing QID 100
Parsing QID 200
Parsing QID 300
Parsing QID 400
Parsing QID 500
Parsing QID 600
Parsing QID 700
Parsing QID 800
Parsing QID 900
Parsing QID 1000
Parsing QID 1100
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for 007 (0/1128)
Missing doc 24549
Missing doc 12773
Missing doc 225130
Missing doc 61917
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for 007 (series) (1/1128)
Missing doc 12773
Missing doc 67479
Missing doc 37106
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for 18th century (2/1128)
Missing doc 164721
Missing doc 67479
Missing doc 13057


Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for afi 100 (movie quotes) (32/1128)
Missing doc 211779
Missing doc 253941
Missing doc 110639
Missing doc 61919
Missing doc 61917
Missing doc 156078
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for africa (33/1128)
Missing doc 10700
Missing doc 67479
Missing doc 61920
Missing doc 206216
Missing doc 225130
Missing doc 37106
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for afterlife (34/1128)
Missing doc 206216
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for a

Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for antarctica (65/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for anti-hero (66/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for anti-semitism (67/1128)
Missing doc 61919
Missing doc 58423
Missing doc 15533
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for anti-war (68/1128)
Missing doc 15738
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]


Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for bad plot (99/1128)
Missing doc 12773
Missing doc 68149
Missing doc 110639
Missing doc 61920
Missing doc 15533
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for bad science (100/1128)
Missing doc 68149
Missing doc 13057
Missing doc 164721
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for bad script (101/1128)
Missing doc 68149
Missing doc 13716
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for bad sequel (102/1128)
Missing doc 133252
Missing doc 24549
Missing

Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for biographical (132/1128)
Missing doc 211779
Missing doc 64699
Missing doc 15738
Missing doc 68149
Missing doc 58423
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for biography (133/1128)
Missing doc 67479
Missing doc 64699
Missing doc 225130
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for biopic (134/1128)
Missing doc 10700
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for birds (135/1128)
Missing doc 9417

...

Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for strippers (974/1128)
Missing doc 13057
Missing doc 94174
Missing doc 15738
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for studio ghibli (975/1128)
Missing doc 164721
Missing doc 110414
Missing doc 110639
Missing doc 12773
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for stunning (976/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for stupid (977/1128)
Missing doc 15738
Missing doc 17882
Missing doc

Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for talking animals (1006/1128)
Missing doc 58423
Missing doc 68149
Missing doc 24549
Missing doc 10700
Missing doc 64699
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for talky (1007/1128)
Missing doc 211779
Missing doc 94174
Missing doc 253768
Missing doc 64699
Missing doc 110414
Missing doc 24549
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for tarantino (1008/1128)
Missing doc 253768
Missing doc 225130
Missing doc 110639
Missing doc 206216
Missing doc 24549
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'

Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for train (1037/1128)
Missing doc 156078
Missing doc 15533
Missing doc 64699
Missing doc 110639
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for trains (1038/1128)
Missing doc 211779
Missing doc 58423
Duplicate Doc in qid:1040 105045
Duplicate Doc in qid:1040 105045
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for transformation (1039/1128)
Missing doc 12773
Missing doc 37106
Missing doc 24549
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for transgender (1040

Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for very good (1071/1128)
Missing doc 164721
Missing doc 15533
Missing doc 110414
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for very interesting (1072/1128)
Missing doc 94174
Missing doc 110639
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for video game (1073/1128)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for video game adaptation (1074/1128)
Missing doc 206216
Missing doc 253768
Missing doc 15533
Missing doc 61919
Missing doc 13057
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{

Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for weird (1103/1128)
Missing doc 225130
Missing doc 37106
Missing doc 110414
Missing doc 13716
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for werewolf (1104/1128)
Missing doc 225130
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for werewolves (1105/1128)
Missing doc 156078
Missing doc 13057
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for western (1106/1128)
Missing doc 13057
Searching tmdb - [{'terms': {'

In [34]:
from ltr import train
trainLog  = train(client,
                  trainingInFile='data/genome_judgments_train.txt',
                  index='tmdb',
                  featureSet='zenit-movies',
                  modelName='zenit-genome')

Running java -jar data/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t DCG@10 -tree 50 -bag 1 -leaf 10 -frate 1.0 -srate 1.0 -train data/genome_judgments_train.txt -save data/zenit-genome_model.txt 
DONE
Delete model zenit-genome: 200
Created Model zenit-genome [Status: 201]


In [35]:
from ltr import search
search(client, "berlin", modelName='zenit-genome')

{"size": 5, "query": {"sltr": {"params": {"keywords": "berlin"}, "model": "zenit-genome"}}}
Searching tmdb - {'size': 5, 'query': [Status: 200]
Berlin Babylon 
3.9911802 
2001 
['Documentary'] 
A documentary focusing on the rebuilding projects in Berlin after the fall of the Berlin Wall. 
---------------------------------------
Berlin: Symphony of a Great City 
3.7912052 
1927 
['Documentary', 'History'] 
Berlin is Walter Ruttmanns black and white documentary of Berlin in 1927. The big city symphony shows a day in the life of metropolitan Berlin. Showing the industrial uprising as well as the lives and working circumstances of the time. 
---------------------------------------
Berlin is in Germany 
3.7785614 
2001 
['Comedy', 'Drama'] 
Berlin is in Germany is a tragic comedy in which an ex-prisoner from east Berlin must find his place in the newly united city. From Director Hannes Stöhr. 
---------------------------------------
Lulu in Berlin 
3.7217476 
1984 
['Documentary'] 
Document